## 1 - Introduction
This notebook is for clean the dataset downloaded from Lending Club website.

## 2 - Dataset

The dataset was download from [Lending Club](https://www.lendingclub.com/statistics/additional-statistics?). It contains information from 2015 to 2019. 

**Data manipulation**: 

    - Concatenate data from 2015 to 2019
    - Remove columns with >20% missing values
    - Save to a new csv file



In [167]:
import pandas as pd

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [168]:
# read data
df_lc = pd.read_csv('./data/data_2015_2019.csv', sep=',',low_memory=True)

In [169]:
df_lc.head()

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,...,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,year
0,68367011,21000.0,21000.0,21000.0,60 months,13.99%,488.53,C,C4,Resident physician,< 1 year,RENT,52000.0,Source Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,credit_card,Credit card refinancing,441xx,OH,14.47,0.0,Jan-2005,...,0.0,3.0,3.0,4.0,4.0,11.0,4.0,4.0,3.0,5.0,0.0,0.0,0.0,1.0,100.0,75.0,0.0,0.0,370357.0,372407.0,23300.0,347057.0,N,N,2015
1,68537655,16800.0,16800.0,16800.0,60 months,12.88%,381.23,C,C2,CEO,10+ years,MORTGAGE,118000.0,Not Verified,Dec-2015,Current,n,https://lendingclub.com/browse/loanDetail.acti...,debt_consolidation,Debt consolidation,636xx,MO,34.29,0.0,Jun-1997,...,12.0,3.0,3.0,4.0,22.0,12.0,5.0,28.0,3.0,11.0,0.0,0.0,0.0,1.0,64.4,25.0,0.0,0.0,412771.0,75808.0,12600.0,115941.0,N,N,2015
2,68356421,22400.0,22400.0,22400.0,60 months,12.88%,508.30,C,C2,Executive Director,6 years,MORTGAGE,95000.0,Not Verified,Dec-2015,Current,n,https://lendingclub.com/browse/loanDetail.acti...,debt_consolidation,Debt consolidation,290xx,SC,22.98,0.0,Apr-1995,...,2.0,3.0,8.0,4.0,5.0,15.0,10.0,15.0,8.0,16.0,0.0,0.0,0.0,3.0,97.0,25.0,0.0,0.0,436841.0,184356.0,21000.0,191682.0,N,N,2015
3,68466926,10000.0,10000.0,10000.0,36 months,6.49%,306.45,A,A2,SERVICE MANAGER,6 years,RENT,85000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,credit_card,Credit card refinancing,160xx,PA,13.07,0.0,Apr-2002,...,0.0,6.0,9.0,7.0,10.0,3.0,13.0,19.0,9.0,14.0,0.0,0.0,0.0,2.0,95.7,28.6,1.0,0.0,61099.0,27957.0,16400.0,30799.0,N,N,2015
4,68616873,8000.0,8000.0,8000.0,36 months,11.48%,263.74,B,B5,Vendor liaison,10+ years,MORTGAGE,42000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,credit_card,Credit card refinancing,029xx,RI,34.80,0.0,Nov-1994,...,1.0,3.0,3.0,3.0,6.0,5.0,5.0,11.0,3.0,8.0,0.0,0.0,0.0,2.0,94.4,33.3,0.0,0.0,256513.0,113782.0,17000.0,135513.0,N,N,2015


In [172]:
df_lc.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2312464 entries, 0 to 2312463
Data columns (total 106 columns):
id                            object
loan_amnt                     float64
funded_amnt                   float64
funded_amnt_inv               float64
term                          object
int_rate                      object
installment                   float64
grade                         object
sub_grade                     object
emp_title                     object
emp_length                    object
home_ownership                object
annual_inc                    float64
verification_status           object
issue_d                       object
loan_status                   object
pymnt_plan                    object
url                           object
purpose                       object
title                         object
zip_code                      object
addr_state                    object
dti                           float64
delinq_2yrs                  

Drop features that are not avaiable to the investors considering a loan investmen
check the description in Data Dictionary.

which including
- feature in loan application
- feature added when the loan was accepted (interest rate or loan grade)

leakage is a situation where a model is trained using the data, while the data will be not available when the model will be use for prediction. 

total payment, loan default, loan early repayment, time of aforementioned event, 
if a loan deault or is paid back early, total payments on the loan will likely be lower. It will make model performance better. 

In [177]:
# check columns

print(sorted(df_lc.columns))

['acc_now_delinq', 'acc_open_past_24mths', 'addr_state', 'all_util', 'annual_inc', 'application_type', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'chargeoff_within_12_mths', 'collection_recovery_fee', 'collections_12_mths_ex_med', 'debt_settlement_flag', 'delinq_2yrs', 'delinq_amnt', 'dti', 'earliest_cr_line', 'emp_length', 'emp_title', 'fico_range_high', 'fico_range_low', 'funded_amnt', 'funded_amnt_inv', 'grade', 'hardship_flag', 'home_ownership', 'id', 'initial_list_status', 'inq_fi', 'inq_last_12m', 'inq_last_6mths', 'installment', 'int_rate', 'issue_d', 'last_credit_pull_d', 'last_fico_range_high', 'last_fico_range_low', 'last_pymnt_amnt', 'last_pymnt_d', 'loan_amnt', 'loan_status', 'max_bal_bc', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc', 'mths_since_rcnt_il', 'mths_since_recent_bc', 'mths_since_recent_inq', 'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_t

In [179]:
# list column name with >20% missing values 
list_drop = ['acc_now_delinq', 'acc_open_past_24mths', 'all_util', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'chargeoff_within_12_mths', 'collection_recovery_fee', 'collections_12_mths_ex_med', 'debt_settlement_flag', 'delinq_2yrs', 'delinq_amnt', 'funded_amnt', 'funded_amnt_inv', 'hardship_flag', 'inq_fi', 'inq_last_12m', 'inq_last_6mths',  'last_credit_pull_d', 'last_fico_range_high', 'last_fico_range_low', 'last_pymnt_amnt', 'last_pymnt_d', 'max_bal_bc', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mths_since_rcnt_il', 'mths_since_recent_bc', 'mths_since_recent_inq', 'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl', 'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats', 'num_tl_120dpd_2m', 'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_il_24m', 'open_rv_12m', 'open_rv_24m', 'out_prncp', 'out_prncp_inv', 'pct_tl_nvr_dlq', 'percent_bc_gt_75', 'policy_code', 'pymnt_plan', 'recoveries',  'tax_liens',  'tot_coll_amt', 'tot_cur_bal', 'tot_hi_cred_lim',  'total_bal_ex_mort', 'total_bal_il', 'total_bc_limit', 'total_cu_tl', 'total_il_high_credit_limit', 'total_pymnt', 'total_pymnt_inv', 'total_rec_int', 'total_rec_late_fee', 'total_rec_prncp', 'total_rev_hi_lim', 'url']

In [180]:
len(list_drop)

74

In [182]:
df_lc.shape

(2312464, 106)

In [162]:
df_all['year'].value_counts(dropna=False)

2019    518115
2018    495250
2017    443587
2016    434415
2015    421097
Name: year, dtype: int64

In [163]:
df_all['term'].value_counts(normalize=True, dropna=False)

 36 months    0.701058
 60 months    0.298927
NaN           0.000015
Name: term, dtype: float64

In [164]:
df_all['loan_status'].value_counts(dropna=False)

Current               1018979
Fully Paid             995328
Charged Off            259485
Late (31-120 days)      22853
In Grace Period         10848
Late (16-30 days)        4683
Issued                    206
Default                    48
NaN                        34
Name: loan_status, dtype: int64

In [153]:
(df_all.isnull().sum()/len(df_all)).sort_values(ascending=False).head(30)

mths_since_rcnt_il       0.197933
all_util                 0.173042
open_acc_6m              0.172897
total_cu_tl              0.172897
inq_last_12m             0.172897
open_act_il              0.172897
max_bal_bc               0.172897
open_rv_24m              0.172897
open_rv_12m              0.172897
total_bal_il             0.172897
open_il_24m              0.172897
open_il_12m              0.172897
inq_fi                   0.172897
mths_since_recent_inq    0.114854
emp_title                0.093810
emp_length               0.074455
num_tl_120dpd_2m         0.035635
mo_sin_old_il_acct       0.030109
bc_util                  0.012270
percent_bc_gt_75         0.011934
bc_open_to_buy           0.011777
mths_since_recent_bc     0.011139
title                    0.010093
last_pymnt_d             0.001433
dti                      0.001250
revol_util               0.000938
avg_cur_bal              0.000054
last_credit_pull_d       0.000030
pct_tl_nvr_dlq           0.000016
inq_last_6mths